In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

# Business Cases for Data Science
## Business Case 5 -  Retail Challenge

### Group AA
**Members**:
- Emil Ahmadov (m20201004@novaims.unl.pt)
- Doris Macean (m20200609@novaims.unl.pt)
- Doyun Shin (m20200565@novaims.unl.pt)
- Anastasiia Tagiltseva (m20200041@novaims.unl.pt)

<a class="anchor" id="0.1"></a>

# **Table of Contents**

1. [Business Understanding](#1)

2. [Data Understanding](#2)

3. [Data Preparation](#3)
  
4. [Exploratory Data Analysis-Quarteraly Analysis](#4)
      
5. [PoS Clusters](#5)
      
6. [Forecast](#6)

# **1. Business Understanding** <a class="anchor" id="1"></a>

- 21 Families of Products  
- 178 Categories of Products
- 1 535 Brands Products
- 8 660 SKUs
- 410 Point-of-Sales
- 1 Jan 2016 -> 1 Nov 2019
- 18.6 Gb of data

**Challenges**

1. Data Analysis and Transformation and some Data/Feature Engineering
2. How can I understand each Point-of-Sale characteristics ?
Quarterly analysis of
Top products sold
Market Share (Family, Category), preferences
Product co-ocorrences
3. Point-of-Sales Clustering
Value
Product preference
4. Units Product forecast (6 weeks ahead)
Units Product forecast by Poit-of-Sale (6 weeks ahead

### Project Plan
| Phase | Time | Resources | Risks |
| :--: | :--------: |:--: | :--------: |
| Business Understanding | 2 days | All analysts | Economic and market changes |
| Data Understanding | 2 days | All analysts | Data problems, technological problems |
| Data Preparation | 2 days | Data scientists, DB engineers | Data problems, technological problems |
| Modeling |4 days | Data scientists | Technological problems, inability to build adequate model |
| Evaluation | 2 days | All analysts | Economic change inability to implement results |
| Deployment | 2 days | Data scientists, DB engineers, implementation team | Economic change inability to implement results |

# **2. Data Understanding** <a class="anchor" id="2"></a>

### Metadata
| Name | Meaning | 
| :--: | :--------|
| ProductFamily_ID | Families of Products|
| ProductCategory_ID | Categories of Products|
| ProductBrand_ID | Brands Products|
| ProductName_ID | Product Name|
| ProductPackSKU_ID | SKU|
| Point-of-Sale_ID | Point-of-Sales|
| Date | Date of Sales|
| Measures | Sell-out units/Sell-out values|
| Value | Value of measure|

# **3. Data Preparation** <a class="anchor" id="3"></a>

In [2]:
import sys
import os
import io

import pandas as pd
import numpy as np
import pickle

# Apriori & Recommendation
import mlxtend as ml
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from sklearn.metrics.pairwise import cosine_similarity

from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder

# Datetime handling
import datetime

# Visualization & Clustering 1
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt

# Visualization & Clustering 2
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import seaborn as sns
color = sns.color_palette()
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure, text
import networkx as nx
import plotly.express as px
from sklearn.cluster import KMeans, AgglomerativeClustering
import plotly.graph_objs as go
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.holtwinters import SimpleExpSmoothing, Holt



# Forecasting
import pmdarima as pm

#Only Applicable for Google Cloud Platform usage.
#from google.cloud import storage
#from io import StringIO
#from io import BytesIO   

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
#Only Applicable for Google Cloud Platform usage.
#client = storage.Client()
#bucket = client.get_bucket('sdymain')
#blob = bucket.get_blob(f"BC5Data_Quantity.pkl")
#bt = blob.download_as_bytes()
#data = pickle.load(BytesIO(bt))

In [4]:
data = pd.read_pickle('BC5Data_Revenue.pkl')

In [5]:
data.head()

ProductFamily_ID  ProductCategory_ID  ProductBrand_ID  ProductName_ID  \
0                 1                  33               35              77   
1                 1                  33              313             657   
2                 1                  33              313             657   
3                 1                  33              313             657   
4                 1                  33              313             657   

        Date  Point-of-Sale_ID  Revenue  
0 2016-12-27               150      495  
1 2016-01-01                13      795  
2 2016-01-01                35      695  
3 2016-01-01                77      470  
4 2016-01-01               104      430

In [6]:
data.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77529406 entries, 0 to 77529405
Data columns (total 7 columns):
 #   Column              Dtype         
---  ------              -----         
 0   ProductFamily_ID    uint64        
 1   ProductCategory_ID  uint64        
 2   ProductBrand_ID     uint64        
 3   ProductName_ID      uint64        
 4   Date                datetime64[ns]
 5   Point-of-Sale_ID    uint64        
 6   Revenue             int32         
dtypes: datetime64[ns](1), int32(1), uint64(5)
memory usage: 3.8 GB


In [7]:
a = data['Point-of-Sale_ID'].unique()
a.sort()
print(a)

[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108
 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126
 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144
 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162
 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180
 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198
 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216
 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234
 235 236 237 238 239 240 241 242 243 244 245 246 24

In [8]:
len(data['Point-of-Sale_ID'].unique()) # Result with above code shows there is no missing PoS number from PoS1 to PoS410

410

In [9]:
data.min()

ProductFamily_ID                        1
ProductCategory_ID                      1
ProductBrand_ID                         1
ProductName_ID                          1
Date                  2016-01-01 00:00:00
Point-of-Sale_ID                        1
Revenue                                 0
dtype: object

In [11]:
data.max()

ProductFamily_ID                       21
ProductCategory_ID                    178
ProductBrand_ID                      1535
ProductName_ID                       2850
Date                  2019-11-01 00:00:00
Point-of-Sale_ID                      410
Revenue                            662575
dtype: object

In [12]:
# Categorical values are also stored as integers temporarily
data['ProductFamily_ID'] = data['ProductFamily_ID'].astype('uint8')
data['ProductCategory_ID'] = data['ProductCategory_ID'].astype('uint8')
data['ProductBrand_ID'] = data['ProductBrand_ID'].astype('uint16')
data['ProductName_ID'] = data['ProductName_ID'].astype('uint16')
data['Point-of-Sale_ID'] = data['Point-of-Sale_ID'].astype('uint16')
data['Revenue'] = data['Revenue'].astype('uint32')

In [13]:
data.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77529406 entries, 0 to 77529405
Data columns (total 7 columns):
 #   Column              Dtype         
---  ------              -----         
 0   ProductFamily_ID    uint8         
 1   ProductCategory_ID  uint8         
 2   ProductBrand_ID     uint16        
 3   ProductName_ID      uint16        
 4   Date                datetime64[ns]
 5   Point-of-Sale_ID    uint16        
 6   Revenue             uint32        
dtypes: datetime64[ns](1), uint16(3), uint32(1), uint8(2)
memory usage: 1.4 GB


In [14]:
counts = data['ProductFamily_ID'].value_counts().sort_values(ascending=False)[:15]

fig = px.bar(x = list(map(str, counts.index)), y = counts.values, labels={'x':'Family ID', 'y':'Count'})
fig.update_layout(showlegend = False)
fig.show()

In [15]:
counts = data['ProductCategory_ID'].value_counts().sort_values(ascending=False)
temp = counts[:15]
temp['Other'] = counts[10:].sum()
counts = temp

fig = px.pie(names = counts.index, values = counts.values, title = 'Product Categories Distribution')
fig.show()

In [16]:
counts = data['ProductBrand_ID'].value_counts().sort_values(ascending=False)[:10]

fig = px.bar(x = list(map(str, counts.index)), y = counts.values, labels={'x':'Brand ID', 'y':'Count'})
fig.update_layout(showlegend = False)
fig.show()

## 3.2 Feature Engineering

In [17]:
%%time
## year month quarter week
data['week'] = pd.DatetimeIndex(data['Date']).week
data['month'] = pd.DatetimeIndex(data['Date']).month
data['quarter'] = pd.DatetimeIndex(data['Date']).quarter
data['year'] = pd.DatetimeIndex(data['Date']).year

CPU times: user 21.4 s, sys: 1.2 s, total: 22.6 s
Wall time: 22.6 s


In [18]:
data.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77529406 entries, 0 to 77529405
Data columns (total 11 columns):
 #   Column              Dtype         
---  ------              -----         
 0   ProductFamily_ID    uint8         
 1   ProductCategory_ID  uint8         
 2   ProductBrand_ID     uint16        
 3   ProductName_ID      uint16        
 4   Date                datetime64[ns]
 5   Point-of-Sale_ID    uint16        
 6   Revenue             uint32        
 7   week                int64         
 8   month               int64         
 9   quarter             int64         
 10  year                int64         
dtypes: datetime64[ns](1), int64(4), uint16(3), uint32(1), uint8(2)
memory usage: 3.8 GB


In [19]:
data.max()

ProductFamily_ID                       21
ProductCategory_ID                    178
ProductBrand_ID                      1535
ProductName_ID                       2850
Date                  2019-11-01 00:00:00
Point-of-Sale_ID                      410
Revenue                            662575
week                                   53
month                                  12
quarter                                 4
year                                 2019
dtype: object

In [20]:
data.min()
#Check Value whether there is actually negative value

ProductFamily_ID                        1
ProductCategory_ID                      1
ProductBrand_ID                         1
ProductName_ID                          1
Date                  2016-01-01 00:00:00
Point-of-Sale_ID                        1
Revenue                                 0
week                                    1
month                                   1
quarter                                 1
year                                 2016
dtype: object

In [21]:
data['week'] = data['week'].astype('int8')
data['month'] = data['month'].astype('int8')
data['quarter'] = data['quarter'].astype('int8')
data['year'] = data['year'] - 2000
data['year'] = data['year'].astype('int8')

In [22]:
data.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77529406 entries, 0 to 77529405
Data columns (total 11 columns):
 #   Column              Dtype         
---  ------              -----         
 0   ProductFamily_ID    uint8         
 1   ProductCategory_ID  uint8         
 2   ProductBrand_ID     uint16        
 3   ProductName_ID      uint16        
 4   Date                datetime64[ns]
 5   Point-of-Sale_ID    uint16        
 6   Revenue             uint32        
 7   week                int8          
 8   month               int8          
 9   quarter             int8          
 10  year                int8          
dtypes: datetime64[ns](1), int8(4), uint16(3), uint32(1), uint8(2)
memory usage: 1.7 GB


# 4. Exploratory Data Analysis-Quarteraly Analysis <a class="anchor" id="4"></a>

In [23]:
counts = data[['Date', 'Revenue']].groupby('Date').count().groupby(pd.Grouper(freq='Q')).sum()
fig = px.scatter(counts['Revenue'], trendline='lowess', labels = {'Revenue':'Count Of Sales', 'Date':'Date By Quarter'}, title = 'Quarterly Analisys of Sales Amount')
fig.update_layout(showlegend = False)
fig.show() 

## 4.1 Top Products

### 4.1.1 Top Family

In [24]:
family = data[['ProductFamily_ID','Point-of-Sale_ID','year','quarter','Revenue']].groupby(['ProductFamily_ID','Point-of-Sale_ID','year','quarter']).sum().reset_index()
family.head()

ProductFamily_ID  Point-of-Sale_ID  year  quarter  Revenue
0                 1                 1    16        1  4998351
1                 1                 1    16        2  5059848
2                 1                 1    16        3  5524699
3                 1                 1    16        4  5128888
4                 1                 1    17        1  4898334

In [25]:
check1 = pd.DataFrame()
for i in family["Point-of-Sale_ID"].unique():
    for q in family["quarter"].unique():
        for y in family["year"].unique():
            check = family[(family["Point-of-Sale_ID"]==i)&(family["quarter"]==q)&(family["year"]==y)]
            check1 = check1.append(check.sort_values(by=["Revenue"],ascending=False).head(5))
check1.sort_values(by=['Point-of-Sale_ID',"year","quarter"], inplace= True)

In [26]:
# Is there 5 families per PoS?
# 4 years * 4 quarters * 5 families = 80, value count for each PoS should be 80.
counts = check1['Point-of-Sale_ID'].value_counts()
counts[counts !=80]

96    65
Name: Point-of-Sale_ID, dtype: int64

In [27]:
del counts
pd.set_option('display.max_rows', 65)
check1[check1['Point-of-Sale_ID']==96]

ProductFamily_ID  Point-of-Sale_ID  year  quarter  Revenue
8077                   2                96    16        1  5988387
53962                  9                96    16        1  5572416
73628                 12                96    16        1  3752724
1520                   1                96    16        1  2681026
128621                21                96    16        1  2613656
53963                  9                96    16        2  5830325
8078                   2                96    16        2  5610593
73629                 12                96    16        2  4070134
128622                21                96    16        2  3544583
1521                   1                96    16        2  2732390
8079                   2                96    16        3  5275152
53964                  9                96    16        3  4852387
73630                 12                96    16        3  3486903
128623                21                96    16        3  3425833
1522                   1                96    16        3  2329823
8080                   2                96    16        4  4719884
53965                  9                96    16        4  4528333
73631                 12                96    16        4  3518977
128624                21                96    16        4  2347143
1523                   1                96    16        4  2296409
8081                   2                96    17        1  4673460
53966                  9                96    17        1  4427827
73632                 12                96    17        1  3367789
128625                21                96    17        1  2467179
1524                   1                96    17        1  2117115
8082                   2                96    17        2  4839625
53967                  9                96    17        2  4429312
128626                21                96    17        2  3575641
73633                 12                96    17        2  3478620
1525                   1                96    17        2  1979457
8083                   2                96    17        3  4915205
53968                  9                96    17        3  4340417
73634                 12                96    17        3  3407869
128627                21                96    17        3  2979450
47411                  8                96    17        3  2086895
8084                   2                96    17        4  4925678
53969                  9                96    17        4  4382769
73635                 12                96    17        4  3360420
128628                21                96    17        4  2915943
47412                  8                96    17        4  2409637
8085                   2                96    18        1  4822335
53970                  9                96    18        1  3929761
73636                 12                96    18        1  3407443
47413                  8                96    18        1  2609667
128629                21                96    18        1  2502414
8086                   2                96    18        2  5019884
53971                  9                96    18        2  3789512
128630                21                96    18        2  3741128
73637                 12                96    18        2  3727568
47414                  8                96    18        2  2804662
8087                   2                96    18        3  5078384
73638                 12                96    18        3  3694326
53972                  9                96    18        3  3551545
128631                21                96    18        3  3282936
47415                  8                96    18        3  2767539
8088                   2                96    18        4  5021075
73639                 12                96    18        4  3996122
53973                  9                96    18        4  3628211
47416                  8                96    18       

In [28]:
# No Sales record after 2019 Q1 for PoS 96. We assume this store is permanently closed, We still keep in our data for reference, but will drop it when necessary, for other analyses.
fam_pos_q_top5 = check1.copy()
del check1

In [29]:
fam_pos_q_top5.to_csv (r'fam_pos_q_top5.csv', index = False, header=True)

In [30]:
test = data[['ProductFamily_ID','ProductBrand_ID','Point-of-Sale_ID','year','quarter','Revenue']].groupby(['ProductFamily_ID','ProductBrand_ID','Point-of-Sale_ID','year','quarter']).sum().reset_index()

# For Brand level breakdown, we concat 
# Alternative method would be adding a new loop line in the next cell below "for y in test['year'].unique() another loop based on Category, and add a new condition to check object, for better presentation, but it takes very long to process the loops."
test = test[test['ProductFamily_ID'].isin(fam_pos_q_top5['ProductFamily_ID'].unique())]
test['F_Brand'] = test['ProductFamily_ID'].astype(str).str.cat(test['ProductBrand_ID'].astype(str), sep='_')
test = test.drop(['ProductFamily_ID','ProductBrand_ID'],axis = 1)
test['B_Revenue'] = test['Revenue']
test = test.drop(['Revenue'],axis = 1)
test.head()

Point-of-Sale_ID  year  quarter F_Brand  B_Revenue
0               150    16        4    1_35        495
1               163    17        2    1_35       1845
2                 1    16        1    1_72      13416
3                 1    16        2    1_72      23478
4                 1    16        3    1_72      14524

In [31]:
fam_brand_pos_q_top5 = pd.DataFrame()
for i in test["Point-of-Sale_ID"].unique():
    for q in test["quarter"].unique():
        for y in test["year"].unique():
            check = test[(test["Point-of-Sale_ID"]==i)&(test["quarter"]==q)&(test["year"]==y)]
            fam_brand_pos_q_top5 = fam_brand_pos_q_top5.append(check.sort_values(by=["B_Revenue"],ascending=False).head(5))
fam_brand_pos_q_top5.sort_values(by=['Point-of-Sale_ID',"year","quarter"], inplace= True)

In [32]:
fam_brand_pos_q_top5[['Family','Brand']] = fam_brand_pos_q_top5['F_Brand'].str.split('_',expand=True)
fam_brand_pos_q_top5 = fam_brand_pos_q_top5.drop('F_Brand',axis = 1)
fam_brand_pos_q_top5.head()

Point-of-Sale_ID  year  quarter  B_Revenue Family Brand
973652                  1    16        1    2637311      9   272
653469                  1    16        1    2595351      2   643
626535                  1    16        1    2524098      2   428
792144                  1    16        1    2143387      7  1049
2705573                 1    16        1    2090547     18   940

In [33]:
fam_brand_pos_q_top5.to_csv ('cat_brand_pos_q_top5.csv')
del fam_brand_pos_q_top5
del test

### 4.1.2 Top Category

In [34]:
category = data[['ProductCategory_ID','Point-of-Sale_ID','year','quarter','Revenue']].groupby(['ProductCategory_ID','Point-of-Sale_ID','year','quarter']).sum().reset_index()
category.head()

ProductCategory_ID  Point-of-Sale_ID  year  quarter  Revenue
0                   1                 1    16        1   288297
1                   1                 1    16        2   197363
2                   1                 1    16        3   150535
3                   1                 1    16        4   275115
4                   1                 1    17        1   302010

In [35]:
# Check Revenue to convert to int
category.max()

ProductCategory_ID          178
Point-of-Sale_ID            410
year                         19
quarter                       4
Revenue               118600229
dtype: int64

In [36]:
# Check Revenue to convert to int
# Check presence of Revenue = 0
category.min()

ProductCategory_ID     1
Point-of-Sale_ID       1
year                  16
quarter                1
Revenue                0
dtype: int64

In [37]:
# Check Revenue to convert to int
test = category[category['Revenue'] % 1 != 0]
test

Empty DataFrame
Columns: [ProductCategory_ID, Point-of-Sale_ID, year, quarter, Revenue]
Index: []

In [38]:
category.Revenue = category.Revenue.astype('int32')

In [39]:
#category = category[category['Point-of-Sale_ID'] != 96]

In [40]:
cat_pos_q_top5 = pd.DataFrame()
for i in category["Point-of-Sale_ID"].unique():
    for q in category["quarter"].unique():
        for y in category["year"].unique():
            check = category[(category["Point-of-Sale_ID"]==i)&(category["quarter"]==q)&(category["year"]==y)]
            cat_pos_q_top5 = cat_pos_q_top5.append(check.sort_values(by=["Revenue"],ascending=False).head(5))
cat_pos_q_top5.sort_values(by=['Point-of-Sale_ID',"year","quarter"], inplace= True)

cat_pos_q_top5.head()

ProductCategory_ID  Point-of-Sale_ID  year  quarter   Revenue
585861                 178                 1    16        1  41654888
577886                 175                 1    16        1   5503062
13114                    3                 1    16        1   1220659
120662                  27                 1    16        1   1119955
96489                   22                 1    16        1    990685

In [41]:
cat_pos_q_top5.to_csv (r'cat_pos_q_top5.csv', index = False, header=True)

In [42]:
cat_pos_q_top5['ProductCategory_ID'].unique()

array([178, 175,   3,  27,  22,  34, 109, 172,   2, 157, 149, 161, 115,
        28,  11,  39, 132, 127, 122, 124, 144, 133, 110,   4,  31, 154,
         1,  44,  37], dtype=uint64)

In [43]:
test = data[['ProductCategory_ID','ProductBrand_ID','Point-of-Sale_ID','year','quarter','Revenue']].groupby(['ProductCategory_ID','ProductBrand_ID','Point-of-Sale_ID','year','quarter']).sum().reset_index()

# For Brand level breakdown, we concat 
# Alternative method would be adding a new loop line in the next cell below "for y in test['year'].unique() another loop based on Category, and add a new condition to check object, for better presentation, but it takes very long to process the loops."
test = test[test['ProductCategory_ID'].isin(cat_pos_q_top5['ProductCategory_ID'].unique())]
test['C_Brand'] = test['ProductCategory_ID'].astype(str).str.cat(test['ProductBrand_ID'].astype(str), sep='_')
test = test.drop(['ProductCategory_ID','ProductBrand_ID'],axis = 1)
test['B_Revenue'] = test['Revenue']
test = test.drop(['Revenue'],axis = 1)
test.head()

Point-of-Sale_ID  year  quarter C_Brand  B_Revenue
0                 1    16        1    1_45       1710
1                 1    16        2    1_45       3040
2                 1    16        3    1_45        760
3                 1    16        4    1_45       2280
4                 1    17        1    1_45       2280

In [44]:
cat_brand_pos_q_top5 = pd.DataFrame()
for i in test["Point-of-Sale_ID"].unique():
    for q in test["quarter"].unique():
        for y in test["year"].unique():
            check = test[(test["Point-of-Sale_ID"]==i)&(test["quarter"]==q)&(test["year"]==y)]
            cat_brand_pos_q_top5 = cat_brand_pos_q_top5.append(check.sort_values(by=["B_Revenue"],ascending=False).head(5))
cat_brand_pos_q_top5.sort_values(by=['Point-of-Sale_ID',"year","quarter"], inplace= True)

In [45]:
cat_brand_pos_q_top5[['Category','Brand']] = cat_brand_pos_q_top5['C_Brand'].str.split('_',expand=True)
cat_brand_pos_q_top5 = cat_brand_pos_q_top5.drop('C_Brand',axis = 1)
cat_brand_pos_q_top5.head()

Point-of-Sale_ID  year  quarter  B_Revenue Category Brand
2402822                 1    16        1    2637311      178   272
2955081                 1    16        1    2595351      178   643
2704287                 1    16        1    2524098      178   428
3659435                 1    16        1    2143387      178  1049
2162295                 1    16        1    2090547      175   940

In [46]:
cat_brand_pos_q_top5.to_csv ('cat_brand_pos_q_top5.csv')
del cat_brand_pos_q_top5
del test

### 4.1.3 Top Product (name)

In [47]:
name = data[['ProductName_ID','Point-of-Sale_ID','year','quarter','Revenue']].groupby(['ProductName_ID','Point-of-Sale_ID','year','quarter']).sum().reset_index()
name.head()

ProductName_ID  Point-of-Sale_ID  year  quarter  Revenue
0               1                 9    19        2     4740
1               1                 9    19        3     9480
2               1                17    19        2    10856
3               1                17    19        3    44768
4               1                17    19        4    11158

In [48]:
#name = name[name['Point-of-Sale_ID'] != 96]

In [49]:
name_pos_q_top5 = pd.DataFrame()
for i in name["Point-of-Sale_ID"].unique():
    for q in name["quarter"].unique():
        for y in name["year"].unique():
            check = name[(name["Point-of-Sale_ID"]==i)&(name["quarter"]==q)&(name["year"]==y)]
            name_pos_q_top5 = name_pos_q_top5.append(check.sort_values(by=["Revenue"],ascending=False).head(5))
name_pos_q_top5.sort_values(by=['Point-of-Sale_ID',"year","quarter"], inplace= True)

name_pos_q_top5.head()

ProductName_ID  Point-of-Sale_ID  year  quarter  Revenue
811568              567                 1    16        1  2637311
1893257            1277                 1    16        1  2595351
1286206             847                 1    16        1  2524098
3088957            1956                 1    16        1  2143387
4408666            2802                 1    16        1  2019794

In [50]:
name_pos_q_top5.to_csv (r'name_pos_q_top5.csv', index = False, header=True)

## 4.2 Market Share (Revenue) and Preferences at Family and Category Level

In [51]:
# Total units of all items sold for each PoS
total_units_pos = family[['Point-of-Sale_ID','year','quarter','Revenue']].groupby(['Point-of-Sale_ID','year','quarter']).sum().reset_index()
total_units_pos.rename(columns={'Revenue': 'PoSTotalSales'}, inplace = True)
total_units_pos.head()

Point-of-Sale_ID  year  quarter  PoSTotalSales
0                 1    16        1       56709204
1                 1    16        2       57707883
2                 1    16        3       61514901
3                 1    16        4       59563711
4                 1    17        1       55997169

### 4.2.2 Market Share Family Level

In [52]:
fam_ms = pd.merge(family, total_units_pos, how='left', left_on=['Point-of-Sale_ID','year','quarter'], right_on = ['Point-of-Sale_ID','year','quarter'])
fam_ms.head()

ProductFamily_ID  Point-of-Sale_ID  year  quarter  Revenue  PoSTotalSales
0                 1                 1    16        1  4998351       56709204
1                 1                 1    16        2  5059848       57707883
2                 1                 1    16        3  5524699       61514901
3                 1                 1    16        4  5128888       59563711
4                 1                 1    17        1  4898334       55997169

In [53]:
#check
print(len(fam_ms))
print(len(family))

133658
133658


In [54]:
# For all
fam_ms['MarketShare'] = fam_ms['Revenue'] / fam_ms['PoSTotalSales']
fam_ms['MarketShare'] = fam_ms['MarketShare'].round(decimals=4)
fam_ms.head()

ProductFamily_ID  Point-of-Sale_ID  year  quarter  Revenue  PoSTotalSales  \
0                 1                 1    16        1  4998351       56709204   
1                 1                 1    16        2  5059848       57707883   
2                 1                 1    16        3  5524699       61514901   
3                 1                 1    16        4  5128888       59563711   
4                 1                 1    17        1  4898334       55997169   

   MarketShare  
0       0.0881  
1       0.0877  
2       0.0898  
3       0.0861  
4       0.0875

In [55]:
# For top 5
fam_ms_top = pd.merge(fam_pos_q_top5, fam_ms.loc[:, fam_ms.columns != 'Revenue'], how='left', left_on=['ProductFamily_ID','Point-of-Sale_ID','year','quarter'], right_on = ['ProductFamily_ID','Point-of-Sale_ID','year','quarter'])
fam_ms_top.head()

ProductFamily_ID  Point-of-Sale_ID  year  quarter   Revenue  PoSTotalSales  \
0                 2                 1    16        1  11262570       56709204   
1                 9                 1    16        1  10606976       56709204   
2                12                 1    16        1   6383352       56709204   
3                18                 1    16        1   5505688       56709204   
4                 1                 1    16        1   4998351       56709204   

   MarketShare  
0       0.1986  
1       0.1870  
2       0.1126  
3       0.0971  
4       0.0881

In [56]:
#check
print(len(fam_pos_q_top5))
print(len(fam_ms_top))

32785
32785


In [57]:
# Check for PoS and quarters that heavily rely on one Product Family >> specialization/diversification choices
fam_ms_top[fam_ms_top['MarketShare']>0.4].sort_values('MarketShare',ascending=False)

ProductFamily_ID  Point-of-Sale_ID  year  quarter   Revenue  \
24810                21               311    17        2  30231033   
24850                21               311    19        2  34264061   
24815                21               311    17        3  27417525   
24835                21               311    18        3  28571571   
24855                21               311    19        3  30998125   
24795                21               311    16        3  23827585   
24830                21               311    18        2  29364703   
24790                21               311    16        2  22388937   
24860                21               311    19        4   8962152   
25450                21               319    17        2  10819601   
24820                21               311    17        4  20337635   
25455                21               319    17        3   9557607   
24840                21               311    18        4  21185197   
24845                21               311    19        1  20446713   
25470                21               319    18        2  10804621   
24825                21               311    18        1  18277415   
24805                21               311    17        1  15754233   
25435                21               319    16        3   8010234   
25475                21               319    18        3   9133462   
25490                21               319    19        2  11397527   
25430                21               319    16        2   8598452   
24800                21               311    16        4  15713545   
7425                 21                93    19        2  23120995   
7430                 21                93    19        3  21941046   
7410                 21                93    18        3  19976993   

       PoSTotalSales  MarketShare  
24810       49358266       0.6125  
24850       57474464       0.5962  
24815       46295056       0.5922  
24835       49415791       0.5782  
24855       53609903       0.5782  
24795       41721434       0.5711  
24830       51645067       0.5686  
24790       41249948       0.5428  
24860       18209768       0.4922  
25450       22742185       0.4758  
24820       43726585       0.4651  
25455       20786885       0.4598  
24840       46558667       0.4550  
24845       45567437       0.4487  
25470       24233607       0.4459  
24825       41180108       0.4438  
24805       35557634       0.4431  
25435       18192872       0.4403  
25475       20776225       0.4396  
25490       25970028       0.4389  
25430       19818449       0.4339  
24800       37080619       0.4238  
7425        54847907       0.4215  
7430        53995278       0.4064  
7410        49677191       0.4021

In [58]:
fam_ms_top.to_csv (r'fam_ms_top.csv', index = False, header=True)

### 4.2.3 Market Share Category Level

In [59]:
cat_ms = pd.merge(category, total_units_pos, how='left', left_on=['Point-of-Sale_ID','year','quarter'], right_on = ['Point-of-Sale_ID','year','quarter'])
cat_ms.head()

ProductCategory_ID  Point-of-Sale_ID  year  quarter  Revenue  PoSTotalSales
0                   1                 1    16        1   288297       56709204
1                   1                 1    16        2   197363       57707883
2                   1                 1    16        3   150535       61514901
3                   1                 1    16        4   275115       59563711
4                   1                 1    17        1   302010       55997169

In [60]:
#check
print(len(cat_ms))
print(len(category))

592418
592418


In [61]:
# For all
cat_ms['MarketShare'] = cat_ms['Revenue'] / cat_ms['PoSTotalSales']
cat_ms['MarketShare'] = cat_ms['MarketShare'].round(decimals=4)
cat_ms.head()

ProductCategory_ID  Point-of-Sale_ID  year  quarter  Revenue  \
0                   1                 1    16        1   288297   
1                   1                 1    16        2   197363   
2                   1                 1    16        3   150535   
3                   1                 1    16        4   275115   
4                   1                 1    17        1   302010   

   PoSTotalSales  MarketShare  
0       56709204       0.0051  
1       57707883       0.0034  
2       61514901       0.0024  
3       59563711       0.0046  
4       55997169       0.0054

In [62]:
# For top 5
cat_ms_top = pd.merge(cat_pos_q_top5, cat_ms.loc[:, cat_ms.columns != 'Revenue'], how='left', left_on=['ProductCategory_ID','Point-of-Sale_ID','year','quarter'], right_on = ['ProductCategory_ID','Point-of-Sale_ID','year','quarter'])
cat_ms_top.head()

ProductCategory_ID  Point-of-Sale_ID  year  quarter   Revenue  \
0                 178                 1    16        1  41654888   
1                 175                 1    16        1   5503062   
2                   3                 1    16        1   1220659   
3                  27                 1    16        1   1119955   
4                  22                 1    16        1    990685   

   PoSTotalSales  MarketShare  
0       56709204       0.7345  
1       56709204       0.0970  
2       56709204       0.0215  
3       56709204       0.0197  
4       56709204       0.0175

In [63]:
#check
print(len(cat_pos_q_top5))
print(len(cat_ms_top))

32785
32785


In [64]:
# Check for PoS and quarters that heavily rely on one Product category >> specialization/diversification choices
# note: as a category can belong to multiple families, the market share value can be much higher.
cat_ms_top[cat_ms_top['MarketShare']>0.4].sort_values('MarketShare',ascending=False)

ProductCategory_ID  Point-of-Sale_ID  year  quarter   Revenue  \
18575                 178               233    17        3  36587427   
230                   178                 3    19        3  68542462   
18595                 178               233    18        3  37871099   
18615                 178               233    19        3  41029264   
18570                 178               233    17        2  35473258   
...                   ...               ...   ...      ...       ...   
25485                 178               319    19        1   9884924   
25475                 178               319    18        3   8974083   
25490                 178               319    19        2  11086113   
25470                 178               319    18        2  10183458   
24785                 178               311    16        1  12915589   

       PoSTotalSales  MarketShare  
18575       41399728       0.8838  
230         77831850       0.8806  
18595       43025657       0.8802  
18615       46635223       0.8798  
18570       40553080       0.8747  
...              ...          ...  
25485       22737788       0.4347  
25475       20776225       0.4319  
25490       25970028       0.4269  
25470       24233607       0.4202  
24785       31291570       0.4127  

[6542 rows x 7 columns]

In [65]:
cat_ms_top[cat_ms_top['MarketShare']>0.4]['ProductCategory_ID'].unique()

array([178], dtype=uint64)

In [66]:
cat_ms_top.to_csv (r'cat_ms_top.csv', index = False, header=True)

**Consideration:**
- ProductCategory_178 is by far the most dominant source of revenue for our client

# 6. Forecast <a class="anchor" id="6"></a>

## 6.1 Units Product Forecast

In [70]:
lifetime_border = data['Date'].max() - datetime.timedelta(weeks = 6)
last_sale = data.groupby('ProductName_ID')['Date'].max()

relevant_products = last_sale[last_sale[last_sale >= lifetime_border].index].index

In [71]:
forecast_data = data[data['ProductName_ID'].isin(relevant_products)][['ProductName_ID','Date', 'Revenue']]
forecast_data = forecast_data.groupby(['ProductName_ID','Date']).sum()
forecast_data.reset_index(inplace=True)

In [72]:
#function to retrieve sales data per product
def prod_data_retriever(prod_id):
    prod_data = forecast_data[forecast_data['ProductName_ID'] == prod_id][['Date', 'Revenue']]
    prod_data.set_index('Date', inplace=True)
    prod_data = prod_data.groupby(pd.Grouper(freq = 'W')).sum()
    prod_data.sort_index(inplace = True)
    return prod_data

In [73]:
prod_data = prod_data_retriever(2609)

In [74]:
fig = px.line(prod_data, labels = {'Revenue':'Units Sold'}, width = 1200)
fig.update_layout(showlegend = False)

fig.show()

In [75]:
prod_data.drop(prod_data.index[0], inplace = True)
prod_data.drop(prod_data.index[-1], inplace = True)

In [76]:
fig = px.histogram(prod_data, title = 'Forecast Data Histogram')
fig.update_layout(showlegend = False)

In [77]:
#fits exponential models
def model_exp(i,j,b,train):
    if j!=None and i!=None:
        model = ExponentialSmoothing(np.asarray(train['Revenue']), trend=i, seasonal=j, seasonal_periods=52, damped=b)
    elif j!=None and i==None:
        model = ExponentialSmoothing(np.asarray(train['Revenue']), trend=i, seasonal=j, seasonal_periods=52)    
    elif j==None and i==None:
        model = ExponentialSmoothing(np.asarray(train['Revenue']), trend=i, seasonal=j)
    else:    
        model = ExponentialSmoothing(np.asarray(train['Revenue']), trend=i, seasonal=j, damped=b)
    return model

In [78]:
params = ['mul', 'add', None]  

In [79]:
#searchs for best parameters for exponential smoothing
def search(params,train,test):
    t = []
    s = []
    d = []
    r2_scores = []
    for i in params:
        for j in params:
            for b in [True, False]:
                model_expon = model_exp(i,j,b,train) 
                fit = model_expon.fit()
                pred = fit.forecast(test.shape[0])
                r2 = r2_score(test['Revenue'], pred)
                t.append(i)
                s.append(j)
                d.append(b)
                r2_scores.append(r2)
    df = pd.DataFrame({'t':t,'s':s, 'd':d, 'r2':r2_scores})
    return df

In [80]:
#forecasts for best model
def forecaster(mod,period,input_series,i,j,b):
    if mod == 'arima':
        smodel = pm.auto_arima(
                            input_series,
                            start_p = 1, start_q = 1,
                            max_q = 5, max_p = 5, m = 12,
                            trace = False)
        forecast = smodel.predict(n_periods = period, return_conf_int = False)
    elif mod == 'ses':
        ses_model = SimpleExpSmoothing(np.asarray(input_series))
        ses_fit = ses_model.fit(smoothing_level=.5)
        forecast = ses_fit.forecast(period)
    elif mod == 'holt':
        holt = Holt(np.asarray(input_series))
        holt_fit = holt.fit(smoothing_level=.3, smoothing_slope=.05)
        forecast = holt_fit.forecast(period)
    else:
        model_expon = model_exp(i,j,b,input_series) 
        fit = model_expon.fit()
        forecast = fit.forecast(period)
    return forecast

In [81]:
#take time series of the product sale data and number of weeks you want to do forecast for as inputs
def forecast(input_series,period):
    #divide data into test and train sets 20/80
    train = input_series[:round(input_series.shape[0]*0.80)]
    test = input_series[round(input_series.shape[0]*0.80):]
    #auto arima
    smodel = pm.auto_arima(
                            train,
                            start_p = 1, start_q = 1,
                            max_q = 5, max_p = 5, m = 12,
                            trace = False)
    #simple exponential smoothing
    ses_model = SimpleExpSmoothing(np.asarray(train['Revenue']))
    ses_fit = ses_model.fit(smoothing_level=.5)
    #holt
    holt = Holt(np.asarray(train['Revenue']))
    holt_fit = holt.fit(smoothing_level=.3, smoothing_slope=.05)
    #exponential smoothing
    exp = search(params,train,test)
    best = exp[exp['r2'] == exp['r2'].max()].values.tolist()[0]
    
    
    #make prediction    
    smodel_forecast = smodel.predict(n_periods = test.shape[0], return_conf_int = False)
    ses_forecast = ses_fit.forecast(test.shape[0])
    holt_forecast = holt_fit.forecast(test.shape[0])

    #r^2 values
    r2_scores = []
    r2_scores.append(r2_score(test['Revenue'], smodel_forecast))
    r2_scores.append(r2_score(test['Revenue'], ses_forecast))
    r2_scores.append(r2_score(test['Revenue'], holt_forecast))
    r2_scores.append(best[3])
    models = ['arima', 'ses', 'holt', 'exp']
    scores=pd.DataFrame({'model':models, 'r2':r2_scores})
    
    #choose the best model
    mod = scores[scores['r2'] == scores['r2'].max()].values.tolist()[0][0]
    r2_final = scores[scores['r2'] == scores['r2'].max()].values.tolist()[0][1]
    
    #forecast for future periods
    future_forecast = forecaster(mod,period,input_series,best[0],best[1],best[2])
    return future_forecast,r2_final,mod

## 6.2 Units Product Forecast by Point-of-Sale 

In [86]:
forecast_data = data[data['ProductName_ID'].isin(relevant_products)][['Point-of-Sale_ID', 'ProductName_ID','Date', 'Revenue']]
forecast_data = forecast_data.groupby(['Point-of-Sale_ID','ProductName_ID','Date']).sum()
forecast_data.reset_index(inplace=True)

In [87]:
#function to retrieve sales data per product
def prod_data_retriever_pos(prod_id, pos_id):
    prod_data = forecast_data[(forecast_data['ProductName_ID'] == prod_id) & (forecast_data['Point-of-Sale_ID'] == pos_id)][['Date', 'Revenue']]
    prod_data.set_index('Date', inplace=True)
    prod_data = prod_data.groupby(pd.Grouper(freq = 'W')).sum()
    prod_data.sort_index(inplace = True)
    return prod_data

In [88]:
prod_data_retriever_pos(2609, 292)

Revenue
Date               
2016-01-03    11154
2016-01-10    81276
2016-01-17   106470
2016-01-24    69262
2016-01-31   114111
...             ...
2019-10-06   151563
2019-10-13   139976
2019-10-20   160471
2019-10-27   143961
2019-11-03    98996

[201 rows x 1 columns]

In [89]:
prod_data = prod_data_retriever_pos(2609,292)
prod_data.drop(prod_data.index[0], inplace = True)
prod_data.drop(prod_data.index[-1], inplace = True)

result = forecast(prod_data,6)

/home/doyun_s411/.local/lib/python3.6/site-packages/statsmodels/tsa/holtwinters/model.py:922: ConvergenceWarning:

Optimization failed to converge. Check mle_retvals.

/home/doyun_s411/.local/lib/python3.6/site-packages/statsmodels/tsa/holtwinters/model.py:922: ConvergenceWarning:

Optimization failed to converge. Check mle_retvals.

/home/doyun_s411/.local/lib/python3.6/site-packages/statsmodels/tsa/holtwinters/model.py:922: ConvergenceWarning:

Optimization failed to converge. Check mle_retvals.

/home/doyun_s411/.local/lib/python3.6/site-packages/statsmodels/tsa/holtwinters/model.py:922: ConvergenceWarning:

Optimization failed to converge. Check mle_retvals.

/home/doyun_s411/.local/lib/python3.6/site-packages/statsmodels/tsa/holtwinters/model.py:922: ConvergenceWarning:

Optimization failed to converge. Check mle_retvals.

/home/doyun_s411/.local/lib/python3.6/site-packages/statsmodels/tsa/holtwinters/model.py:922: ConvergenceWarning:

Optimization failed to converge. Check mle_re

In [90]:
result

(array([164053.06426971, 163406.85639728, 164686.56834264, 164336.39104095,
        165690.00305974, 161288.66030578]),
 0.033963438288255654,
 'arima')

In [91]:
r2_top5=[]
for i in top_5:
    prod_data = prod_data_retriever_pos(i, 292)
    prod_data.drop(prod_data.index[0], inplace = True)
    prod_data.drop(prod_data.index[-1], inplace = True)
    result = forecast(prod_data,6)
    r2_top5.append(result[1])

/home/doyun_s411/.local/lib/python3.6/site-packages/statsmodels/tsa/holtwinters/model.py:922: ConvergenceWarning:

Optimization failed to converge. Check mle_retvals.

/home/doyun_s411/.local/lib/python3.6/site-packages/statsmodels/tsa/holtwinters/model.py:922: ConvergenceWarning:

Optimization failed to converge. Check mle_retvals.

/home/doyun_s411/.local/lib/python3.6/site-packages/statsmodels/tsa/holtwinters/model.py:922: ConvergenceWarning:

Optimization failed to converge. Check mle_retvals.

/home/doyun_s411/.local/lib/python3.6/site-packages/statsmodels/tsa/holtwinters/model.py:922: ConvergenceWarning:

Optimization failed to converge. Check mle_retvals.

/home/doyun_s411/.local/lib/python3.6/site-packages/statsmodels/tsa/holtwinters/model.py:922: ConvergenceWarning:

Optimization failed to converge. Check mle_retvals.

/home/doyun_s411/.local/lib/python3.6/site-packages/statsmodels/tsa/holtwinters/model.py:922: ConvergenceWarning:

Optimization failed to converge. Check mle_re

/home/doyun_s411/.local/lib/python3.6/site-packages/statsmodels/tsa/holtwinters/model.py:922: ConvergenceWarning:

Optimization failed to converge. Check mle_retvals.

/home/doyun_s411/.local/lib/python3.6/site-packages/statsmodels/tsa/holtwinters/model.py:922: ConvergenceWarning:

Optimization failed to converge. Check mle_retvals.

/home/doyun_s411/.local/lib/python3.6/site-packages/statsmodels/tsa/holtwinters/model.py:922: ConvergenceWarning:

Optimization failed to converge. Check mle_retvals.

/home/doyun_s411/.local/lib/python3.6/site-packages/statsmodels/tsa/holtwinters/model.py:80: RuntimeWarning:

overflow encountered in matmul

/home/doyun_s411/.local/lib/python3.6/site-packages/statsmodels/tsa/holtwinters/model.py:922: ConvergenceWarning:

Optimization failed to converge. Check mle_retvals.

/home/doyun_s411/.local/lib/python3.6/site-packages/statsmodels/tsa/holtwinters/model.py:922: ConvergenceWarning:

Optimization failed to converge. Check mle_retvals.

/home/doyun_s411/.

In [92]:
r2_top5

[-0.00042603049111922253,
 0.033963438288255654,
 0.7340317688995776,
 0.7259861880164573,
 0.7088647559232185]

In [93]:
sum(r2_top5)/len(r2_top5)

0.44048402412727794

In [94]:
top_5

[1277, 2609, 481, 993, 1147]

# Utility

In [ ]:
# Memory Usage check when needed
objects=[]
for name,obj in locals().items():
    objects.append([name,sys.getsizeof(obj)])
sorted(objects,key=lambda x: x[1],reverse=True)

In [ ]:
# Total Memory Usage
import os, psutil
process = psutil.Process(os.getpid())
print(process.memory_info().rss/(1024**3))  # in gb